In [1]:
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
from xgboost import XGBClassifier
from scipy.signal import find_peaks
import shap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import plotly.graph_objs as go
# from plotly.subplots import make_subplots
import duckdb
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)  # Ignore known FutureWarnings (e.g., pandas Int64Index)

# Global DuckDB connection (persists between function calls)
con = duckdb.connect(database=':memory:')
start_date = (datetime.today() - timedelta(days=365*10)).strftime('%Y-%m-%d')
end_date = datetime.today().strftime('%Y-%m-%d')
symbol = 'RYCEY'  # Default ticker for testing

def get_date_range(df, date_col='Date'):
    """Safely get first and last dates from DataFrame"""
    # Get dates from column or index
    dates = df[date_col].values if date_col in df.columns else df.index.values
    
    # Convert to datetime and handle empty cases
    dates = pd.to_datetime(dates)
    if len(dates) == 0:
        return None, None
    
    # Use iloc-style access to avoid index issues
    first_date = dates[0].date()
    last_date = dates[-1].date() if len(dates) > 1 else first_date
    
    return first_date, last_date

def calculate_dominant_periods(df, window_size=60):
    """
    Calculate dominant periods using FFT with bias reduction techniques:
    - Proper windowing (Hamming window)
    - Detrending
    - Advanced peak detection
    - Noise floor consideration
    """
    # Initialize results
    dominant_periods = np.full(len(df), np.nan)
    
    # Create window function
    window = np.hamming(window_size)
    
    # Minimum meaningful period (avoid detecting very high frequencies)
    min_period = 2  # Minimum period of 2 samples
    
    for i in range(window_size, len(df)):
        # Get window data and detrend
        window_data = df['Close'].iloc[i - window_size:i].values
        window_data = window_data - np.mean(window_data)  # Remove DC component
        
        # Apply window function
        windowed_data = window_data * window
        
        # Perform FFT
        fft = np.fft.fft(windowed_data)
        freqs = np.fft.fftfreq(len(fft))
        
        # Only consider positive frequencies
        positive_mask = freqs > 0
        positive_freqs = freqs[positive_mask]
        fft_mag = np.abs(fft[positive_mask])
        
        # Convert to power spectrum (magnitude squared)
        power_spectrum = fft_mag**2
        
        # Find all peaks in the power spectrum
        peaks, properties = find_peaks(power_spectrum, height=0)
        
        if len(peaks) > 0:
            # Calculate noise floor (median of non-peak values)
            noise_floor = np.median(power_spectrum[np.setdiff1d(
                np.arange(len(power_spectrum)), peaks
            )])
            
            # Filter peaks that are significantly above noise floor (3dB threshold)
            significant_peaks = peaks[properties['peak_heights'] > 2*noise_floor]
            
            if len(significant_peaks) > 0:
                # Get top 3 significant peaks by magnitude
                top_peaks = significant_peaks[
                    np.argsort(power_spectrum[significant_peaks])[-3:]
                ]
                
                # Select the peak with highest frequency among top peaks
                # (avoids always selecting the lowest frequency peak)
                selected_peak = top_peaks[
                    np.argmax(positive_freqs[top_peaks])
                ]
                
                dom_freq = positive_freqs[selected_peak]
                dom_period = abs(1/dom_freq)
                
                # Validate period is within reasonable bounds
                if dom_period < min_period or dom_period > window_size:
                    dom_period = np.nan
            else:
                dom_period = np.nan
        else:
            dom_period = np.nan
        
        dominant_periods[i] = dom_period
    
    # Post-processing
    df['Dominant_Period'] = dominant_periods
    
    # Linear interpolation for missing values (better than forward fill)
    df['Dominant_Period'] = df['Dominant_Period'].interpolate(
        method='linear',
        limit_area='inside'  # Only fill between valid values
    )
    
    # Optional: Smooth the final series
    df['Dominant_Period'] = df['Dominant_Period'].rolling(
        window=3,
        center=True,
        min_periods=1
    ).mean()
    
    return df


In [96]:
#SIMPLE DATA PULL
def get_stock_data(ticker: str) -> pd.DataFrame:
    with duckdb.connect() as con:
        if Path(f'data/{ticker}.parquet').exists():
            pass
            print(f"Loading data for {ticker} from local Parquet file.")
            return con.execute(f"SELECT * FROM read_parquet('data/{ticker}.parquet')").df()
        else:
            print(f"Downloading data for {ticker} from Yahoo Finance.")
            df = yf.download(ticker, start=start_date, end=end_date, progress=False)
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            df.to_parquet(f'data/{ticker}.parquet')
            return df
df = get_stock_data(symbol)  # Example usage to ensure data is downloaded

#type(df)

Loading data for RYCEY from local Parquet file.


In [ ]:
def clean_column_names(df, ticker):
    """Flatten multi-index columns and remove ticker from column names"""
    # Convert columns to list if they're in the problematic format
    if any(isinstance(col, tuple) for col in df.columns):
        new_columns = []
        for col in df.columns:
            if isinstance(col, tuple):
                # Keep just the metric name (e.g., 'Close') and drop the ticker
                new_columns.append(col[0])
            else:
                new_columns.append(col)
        df.columns = new_columns
    return df

def get_stock_data(ticker: str, force_download: bool = False) -> duckdb.DuckDBPyRelation:
    """Fetch stock data from cache or Yahoo Finance, storing in DuckDB."""
    Path('data').mkdir(exist_ok=True)
    parquet_path = f'data/{ticker}.parquet'
    
    if not force_download and Path(parquet_path).exists():
        print(f"Loading cached data for {ticker}")
        con.execute(f"CREATE OR REPLACE TABLE {ticker}_data AS SELECT * FROM read_parquet('{parquet_path}')").df()
    else:
        print(f"Downloading fresh data for {ticker}")
        df = yf.download(
            ticker, 
            start=(datetime.today() - timedelta(days=365*10)).strftime('%Y-%m-%d'),
            end=datetime.today().strftime('%Y-%m-%d'),
            progress=False
        )
        
        # Clean the column names
        df = clean_column_names(df, ticker)
        
        # Select our standard columns and reset index
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']].reset_index()
        
        # Save to Parquet
        df.to_parquet(parquet_path, index=False)
        
        # Register with DuckDB
        con.execute(f"CREATE OR REPLACE TABLE {ticker}_data AS SELECT * FROM df")
    
    return con.table(f'{ticker}_data').df()

# Usage
if __name__ == "__main__":
    # Get data
    df = get_stock_data("RYCEY")
    
    # Show schema to verify clean column names
    print("Table Schema:")
    print(con.execute("DESCRIBE rycey_data").df())
    
    # Query recent data
    recent_data = con.execute("""
        SELECT 
            MIN(Date) as start_date
            ,MAX(Date) as end_date
            ,COUNT(*) as row_count
            ,AVG(Close) as avg_close
        FROM df
    """).df()
    
    print("\nRecent Data:")
    print(recent_data)
    print("\nColumn Names:", recent_data.columns.tolist())

Loading cached data for RYCEY
Table Schema:
  column_name   column_type null   key default extra
0        Date  TIMESTAMP_NS  YES  None    None  None
1        Open        DOUBLE  YES  None    None  None
2        High        DOUBLE  YES  None    None  None
3         Low        DOUBLE  YES  None    None  None
4       Close        DOUBLE  YES  None    None  None
5      Volume        BIGINT  YES  None    None  None

Recent Data:
  start_date   end_date  row_count  avg_close
0 2015-06-05 2025-05-30       2512   6.510762

Column Names: ['start_date', 'end_date', 'row_count', 'avg_close']


In [4]:
# Technical Indicators & Advanced Metrics

# 2.1 Relative Strength Index (RSI) - 14 day
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
rs = avg_gain / avg_loss
df['RSI14'] = 100 - (100 / (1 + rs))

# 2.2 Moving Average Convergence/Divergence (MACD: 12,26 with Signal 9)
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD_line'] = ema12 - ema26
df['MACD_signal'] = df['MACD_line'].ewm(span=9, adjust=False).mean()
df['MACD_hist'] = df['MACD_line'] - df['MACD_signal']

# 2.3 Average True Range (ATR) - 14 day
high_low = df['High'] - df['Low']
high_prev_close = (df['High'] - df['Close'].shift(1)).abs()
low_prev_close  = (df['Low'] - df['Close'].shift(1)).abs()
true_range = pd.DataFrame({'hl': high_low, 'hc': high_prev_close, 'lc': low_prev_close}).max(axis=1)
df['ATR14'] = true_range.rolling(window=14).mean()

# 2.4 Bollinger Bands (20-day SMA ± 2 STD)
rolling_mean20 = df['Close'].rolling(window=20).mean()
rolling_std20  = df['Close'].rolling(window=20).std()
df['BB_mid']   = rolling_mean20
df['BB_upper'] = rolling_mean20 + 2 * rolling_std20
df['BB_lower'] = rolling_mean20 - 2 * rolling_std20

# 2.5 On-Balance Volume (OBV)
direction = np.sign(df['Close'].diff().fillna(0))
df['OBV'] = (direction * df['Volume']).cumsum()

# 2.6 Stochastic Oscillator %K and %D (14-day)
window = 14
lowest_low  = df['Low'].rolling(window).min()
highest_high = df['High'].rolling(window).max()
df['%K'] = (df['Close'] - lowest_low) / (highest_high - lowest_low + 1e-9) * 100  # +1e-9 to avoid zero division
df['%D'] = df['%K'].rolling(3).mean()

# 2.7 Money Flow Index (MFI) - 14 day
typical_price = (df['High'] + df['Low'] + df['Close']) / 3.0
mf = typical_price * df['Volume']
tp_diff = typical_price.diff()
pos_mf = mf.where(tp_diff > 0, 0.0).rolling(window).sum()
neg_mf = mf.where(tp_diff < 0, 0.0).rolling(window).sum()
df['MFI14'] = 100 - 100 / (1 + pos_mf / (neg_mf + 1e-9))

# 2.8 Commodity Channel Index (CCI) - 20 day
TP20 = typical_price.rolling(20).mean()
MD20 = (typical_price - TP20).abs().rolling(20).mean()  # Mean deviation
df['CCI20'] = (typical_price - TP20) / (0.015 * MD20)

# 2.9 Williams %R (14-day)
df['Williams_%R'] = (highest_high - df['Close']) / (highest_high - lowest_low + 1e-9) * -100

# 2.10 Rate of Change (ROC) - 10 day
df['ROC10'] = df['Close'].pct_change(periods=10) * 100

# 2.11 GARCH(1,1) Volatility Estimate (daily)
returns = df['Close'].pct_change().fillna(0)
# Initialize GARCH parameters (omega, alpha, beta) and variance
var0 = returns.var()
alpha, beta = 0.1, 0.85
omega = var0 * max(0, (1 - alpha - beta))
garch_vars = [var0]
for r in returns.iloc[1:]:
    new_var = omega + alpha * (r**2) + beta * garch_vars[-1]
    garch_vars.append(new_var)
df['GARCH_vol'] = np.sqrt(garch_vars)

# 2.12 Fourier Transform Dominant Period
# Compute the Fourier Transform dominant period for each row using a rolling window
df = calculate_dominant_periods(df, window_size=60)

# 2.13 One-day Return (%) as feature
df['Return1'] = returns * 100

# Drop initial rows with NaN values from rolling calculations
df.dropna(inplace=True)
print(f"After feature engineering: {len(df)} data points, {df.shape[1]} columns (incl. features).")
print(f"{df.Dominant_Period.describe()}\n{df.Close.describe()}")
df.sample(10)  # display last 5 rows of the last 10 feature columns

After feature engineering: 2453 data points, 24 columns (incl. features).
count    2453.000000
mean        4.803999
std         1.820771
min         2.142857
25%         3.537582
50%         4.615385
75%         5.555556
max        16.111111
Name: Dominant_Period, dtype: float64
count    2453.000000
mean        6.387663
std         3.950234
min         0.744062
25%         1.865117
50%         7.301733
75%         9.914142
max        13.825971
Name: Close, dtype: float64


,Date,Open,High,Low,Close,Volume,RSI14,MACD_line,MACD_signal,MACD_hist,...,OBV,%K,%D,MFI14,CCI20,Williams_%R,ROC10,GARCH_vol,Dominant_Period,Return1
1834,2022-09-16,0.892875,0.892875,0.863112,0.873033,3351900,35.071058,-0.040442,-0.040748,0.000307,...,-252834700.0,31.249980,35.906841,44.777387,-48.016808,-68.750020,1.149419,0.028509,2.222222,-2.222223
586,2017-10-02,11.159010,11.196551,11.121469,11.187165,15400,52.011253,0.005713,0.014286,-0.008573,...,-2786300.0,60.869660,55.555698,54.848986,17.402721,-39.130340,-0.334453,0.020977,8.888889,0.421234
1905,2022-12-28,1.091292,1.091292,1.061529,1.061529,2766900,53.072743,0.011843,0.018514,-0.006671,...,-231514900.0,36.363692,36.363692,43.815725,-0.382888,-63.636307,-2.727273,0.022624,4.984127,0.000000
1943,2023-02-23,1.557571,1.577412,1.517887,1.547650,13195400,76.210061,0.039499,0.030767,0.008733,...,-196738400.0,90.624973,56.134222,62.838888,391.774194,-9.375027,19.999996,0.066844,5.166667,19.999996
789,2018-07-24,12.561286,12.608831,12.523250,12.570795,21300,59.937571,0.233960,0.293385,-0.059425,...,-1890300.0,41.304717,36.047365,58.464115,7.490571,-58.695283,-1.636889,0.019789,4.743590,0.685465
663,2018-01-23,11.476885,11.580620,11.467454,11.580620,23200,61.401362,0.187983,0.103324,0.084659,...,-4298000.0,80.291914,81.751782,44.054249,110.098074,-19.708086,4.067795,0.026592,3.157895,0.244892
854,2018-10-24,10.507358,10.621465,10.212581,10.241109,139100,25.391933,-0.461111,-0.366066,-0.095045,...,-2879800.0,1.515187,4.745237,13.482042,-115.813111,-98.484813,-7.869966,0.025777,2.749907,-2.972970
473,2017-04-21,9.614589,9.679741,9.586667,9.679741,27700,67.278363,0.235941,0.197268,0.038672,...,-6289600.0,92.592719,88.271721,92.166698,101.325597,-7.407281,6.122475,0.021006,5.333333,0.192667
2431,2025-02-04,7.281892,7.321575,7.262050,7.271971,1570500,52.784590,0.040722,0.029939,0.010783,...,31299000.0,67.948730,69.658133,71.542298,46.554276,-32.051270,0.273598,0.022011,10.000000,0.686810
139,2015-12-22,7.660381,7.761055,7.632925,7.761055,167900,40.156253,-0.232294,-0.261173,0.028879,...,-6169700.0,29.464223,35.416557,20.019450,-100.147786,-70.535777,-4.611942,0.026134,5.538462,-0.586169


In [5]:
# Define target: 1 if next day's Close is higher than today's, else 0
df['UpNext'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)  # drop last row (no target for it)
print("Target 'UpNext':", df['UpNext'].value_counts().to_dict())  # distribution of up/down


Target 'UpNext': {0: 1291, 1: 1162}


In [7]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'RSI14', 'MACD_line',
       'MACD_signal', 'MACD_hist', 'ATR14', 'BB_mid', 'BB_upper', 'BB_lower',
       'OBV', '%K', '%D', 'MFI14', 'CCI20', 'Williams_%R', 'ROC10',
       'GARCH_vol', 'Dominant_Period', 'Return1', 'UpNext'],
      dtype='object')

In [8]:
# Train-test split (80% train, 20% test by date order)
train_size = int(0.8 * len(df))
train_df = df.iloc[:train_size].copy()
test_df  = df.iloc[train_size:].copy()
train_start, train_end = get_date_range(train_df, 'Date')
test_start, test_end = get_date_range(test_df, 'Date')

print(f"Training: {train_start} to {train_end} ({len(train_df)} samples)")
print(f"Testing: {test_start} to {test_end} ({len(test_df)} samples)")

# Features for modeling (20+ features we engineered)
feature_cols = [
    'RSI14','MACD_line','MACD_signal','MACD_hist','ATR14',
    'BB_mid','BB_upper','BB_lower','OBV','%K','%D',
    'MFI14','CCI20','Williams_%R','ROC10','GARCH_vol',
    'Dominant_Period','Return1','Close','Volume'
]
X_train = train_df[feature_cols]
y_train = train_df['UpNext'].astype(int)
X_test  = test_df[feature_cols]
y_test  = test_df['UpNext'].astype(int)


Training: 2015-08-28 to 2023-06-14 (1962 samples)
Testing: 2023-06-15 to 2025-05-30 (491 samples)


In [9]:
# 4.1 Train XGBoost model
xgb_model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
accuracy = (y_pred == y_test).mean()
print(f"XGBoost Test Accuracy: {accuracy:.2%}")

# 4.2 SHAP feature importance for XGBoost
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)
# If XGBoost classifier returns a list (one per class), take the second element (positive class)
if isinstance(shap_values, list):
    shap_values = shap_values[1]  # shap values for class "1"
# Calculate mean absolute SHAP value for each feature
importance = np.mean(np.abs(shap_values), axis=0)
feature_importance = pd.Series(importance, index=X_test.columns).sort_values(ascending=False)
print("Top 5 features by SHAP importance:")
print(feature_importance.head(5))


XGBoost Test Accuracy: 50.10%
Top 5 features by SHAP importance:
RSI14      0.386637
OBV        0.372639
Return1    0.300512
MFI14      0.245748
%D         0.245400
dtype: float32


In [10]:
# 5.1 Prepare sequence data for Transformer
window_size = 60  # sequence length (days)
X_values = df[feature_cols].values
y_values = df['UpNext'].values.astype(int)

X_seq, y_seq = [], []
for i in range(window_size, len(X_values)):
    # Sequence of features for days [i-window_size ... i-1]
    X_seq.append(X_values[i-window_size:i])
    # Label for sequence = UpNext of day i-1 (predicting day i relative to i-1)
    y_seq.append(y_values[i-1])
X_seq = np.array(X_seq, dtype=np.float32)
y_seq = np.array(y_seq, dtype=np.int64)

# Split sequence data into train and test sets corresponding to original split
# A sequence ending at index j (label index j) belongs to train if j < train_size, else test
train_seq_idx = np.where((np.arange(window_size, len(X_values)) - 1) < train_size)[0]
test_seq_idx  = np.where((np.arange(window_size, len(X_values)) - 1) >= train_size)[0]
X_seq_train = X_seq[train_seq_idx]
y_seq_train = y_seq[train_seq_idx]
X_seq_test  = X_seq[test_seq_idx]
y_seq_test  = y_seq[test_seq_idx]
print(f"Sequences: {X_seq_train.shape[0]} train sequences, {X_seq_test.shape[0]} test sequences.")

# Convert to torch tensors
X_seq_train_t = torch.tensor(X_seq_train)
y_seq_train_t = torch.tensor(y_seq_train)
X_seq_test_t  = torch.tensor(X_seq_test)
y_seq_test_t  = torch.tensor(y_seq_test)


Sequences: 1903 train sequences, 490 test sequences.


In [11]:
# 5.2 Define Transformer model (encoder) for binary classification
class StockTransformer(nn.Module):
    def __init__(self, input_features, d_model=64, nhead=4, num_layers=2, num_classes=2):
        super(StockTransformer, self).__init__()
        self.input_features = input_features
        self.d_model = d_model
        # Feature embedding layer: project input features to d_model dimensions
        self.feature_embed = nn.Linear(input_features, d_model)
        # Transformer Encoder layers
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Final output layer
        self.fc_out = nn.Linear(d_model, num_classes)
    def forward(self, x):
        # x shape: (batch, seq_len, input_features)
        x = x.permute(1, 0, 2)               # -> (seq_len, batch, input_features)
        x = self.feature_embed(x)            # -> (seq_len, batch, d_model)
        x = self.transformer_encoder(x)      # -> (seq_len, batch, d_model)
        out = x[-1, :, :]                    # take the last time step's output: (batch, d_model)
        out = self.fc_out(out)               # -> (batch, num_classes)
        return out

# Initialize model, loss, optimizer
model = StockTransformer(input_features=X_seq_train.shape[2], d_model=64, nhead=4, num_layers=2, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [12]:
# 5.3 Train the Transformer model
epochs = 5
batch_size = 32
train_dataset = TensorDataset(X_seq_train_t, y_seq_train_t)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    avg_loss = total_loss / len(train_dataset)
    print(f"Epoch {epoch}/{epochs} - Training Loss: {avg_loss:.4f}")

# 5.4 Evaluate on test sequences
model.eval()
with torch.no_grad():
    test_outputs = model(X_seq_test_t)
    test_preds = test_outputs.argmax(dim=1).numpy()
test_accuracy = (test_preds == y_seq_test).mean()
print(f"Transformer Test Accuracy: {test_accuracy:.2%}")


Epoch 1/5 - Training Loss: 0.7312
Epoch 2/5 - Training Loss: 0.7002
Epoch 3/5 - Training Loss: 0.6939
Epoch 4/5 - Training Loss: 0.6961
Epoch 5/5 - Training Loss: 0.6948
Transformer Test Accuracy: 47.55%


In [105]:
# 6.1 Ensemble probabilities (average of XGBoost and Transformer)
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]            # probability of class 1 from XGBoost
trans_proba = F.softmax(test_outputs, dim=1).numpy()[:, 1]   # probability of class 1 from Transformer
# Ensure we align lengths (Transformer test may have one fewer if sequence window covers until second-last day)
min_len = min(len(xgb_proba), len(trans_proba))
ensemble_proba = (xgb_proba[:min_len] + trans_proba[:min_len]) / 2

# 6.2 Compute Kelly fraction for the latest day in test
latest_p = ensemble_proba[-1]   # ensemble probability of up for the most recent day in test set
kelly_fraction = 2 * latest_p - 1
print(f"Latest ensemble 'Up' probability: {latest_p:.2%}")
print(f"Kelly fraction: {kelly_fraction:.2f}")

# 6.3 Fuzzy logic verdict based on Kelly fraction
if kelly_fraction > 0.5:
    verdict = "Strong Buy"
elif kelly_fraction > 0.1:
    verdict = "Buy"
elif kelly_fraction < -0.5:
    verdict = "Strong Sell"
elif kelly_fraction < -0.1:
    verdict = "Sell"
else:
    verdict = "Hold/Neutral"

print("Fuzzy Verdict for the latest day:", verdict)


Latest ensemble 'Up' probability: 25.36%
Kelly fraction: -0.49
Fuzzy Verdict for the latest day: Sell


In [115]:
df.index = pd.to_datetime(df['Date'])

In [1]:
import pandas as pd
import duckdb
import json
from pathlib import Path

def load_and_save_results(db_path="flint/results/audit_log.duckdb", output_csv="audit_log_results.csv"):
    """
    Loads all analysis results from the DuckDB database, saves them to a CSV,
    and returns them as a pandas DataFrame.

    Args:
        db_path (str): The path to the DuckDB database file.
        output_csv (str): The filename for the output CSV file.

    Returns:
        pandas.DataFrame: A DataFrame containing all analysis results,
                          or an empty DataFrame if an error occurs.
    """
    db_file = Path(db_path)
    if not db_file.exists():
        print(f"Error: Database file not found at '{db_path}'")
        return pd.DataFrame()

    print(f"Connecting to '{db_path}'...")
    with duckdb.connect(database=str(db_file), read_only=True) as con:
        try:
            # Fetch the entire table directly into a pandas DataFrame
            df = con.execute("SELECT * FROM analysis_results").fetchdf()
            print(f"Successfully loaded {len(df)} records.")
        except duckdb.Error as e:
            print(f"An error occurred while querying the database: {e}")
            return pd.DataFrame()

    if df.empty:
        print("The 'analysis_results' table is empty.")
        return df

    # Save the full, raw DataFrame to a CSV file
    df.to_csv(output_csv, index=False)
    print(f"All {len(df)} records have been saved to '{output_csv}'.")
    
    return df

# --- Usage in your Jupyter Notebook cell ---

# 1. Call the function to get your data
results_df = load_and_save_results()

# 2. Display the first few rows of the main DataFrame
#    In Jupyter, this will automatically render as a nice HTML table.
results_df.head()

Connecting to 'flint/results/audit_log.duckdb'...
Successfully loaded 2 records.
All 2 records have been saved to 'audit_log_results.csv'.


,run_id,ticker,execution_timestamp,model_name,git_hash,run_config,predictions,metrics,shap_importance
0,1749616111801812992,RYCEY,2025-06-11 04:28:31.801836,XGBoost_v1,cd783a9b3650f05b7df924d7b2f526006dfe3e5b,"{""n_estimators"": 100, ""random_state"": 42, ""eva...","{""probabilities"": [0.4838925302028656, 0.91009...","{""accuracy"": 0.505091649694501, ""kelly_fractio...","{""features"": [""RSI14"", ""MACD_line"", ""MACD_sign..."
1,1749728587365390080,RYCEY,2025-06-12 11:43:07.365401,XGBoost_v1,cd783a9b3650f05b7df924d7b2f526006dfe3e5b,"{""n_estimators"": 100, ""random_state"": 42, ""eva...","{""probabilities"": [0.8327372074127197, 0.36963...","{""accuracy"": 0.483739837398374, ""kelly_fractio...","{""features"": [""RSI14"", ""MACD_line"", ""MACD_sign..."


In [4]:
from IPython.display import display, HTML

def inspect_run_details(df, index):
    """
    Selects a single run by its index and displays its nested JSON columns
    (metrics, predictions, shap_importance) in a readable format.

    Args:
        df (pandas.DataFrame): The DataFrame containing all results.
        index (int): The integer index of the row you want to inspect.
    """
    if index not in df.index:
        print(f"Error: Index {index} is out of bounds. Please choose an index between 0 and {len(df)-1}.")
        return

    # Select the specific run (row) you want to inspect
    run_data = df.loc[index]
    
    print("-" * 50)
    display(HTML(f"<h3>Inspecting Run for Ticker: {run_data['ticker']} (Index: {index})</h3>"))
    print(f"Timestamp: {run_data['execution_timestamp']}")
    print("-" * 50)

    # --- 1. Unpack and display METRICS ---
    display(HTML("<h4>Metrics</h4>"))
    try:
        metrics_dict = json.loads(run_data['metrics'])
        metrics_series = pd.Series(metrics_dict, name="Value")
        display(metrics_series.to_frame())
    except (json.JSONDecodeError, TypeError):
        print("Could not parse the 'metrics' column.")

    # --- 2. Unpack and display PREDICTIONS ---
    display(HTML("<h4>Predictions (first 20 probabilities)</h4>"))
    try:
        predictions_dict = json.loads(run_data['predictions'])
        # The probabilities are a list inside the dictionary
        if 'probabilities' in predictions_dict:
            predictions_df = pd.DataFrame(predictions_dict['probabilities'], columns=['probability'])
            display(predictions_df.head(20))
        else:
            print("'probabilities' key not found in predictions JSON.")
    except (json.JSONDecodeError, TypeError):
        print("Could not parse the 'predictions' column.")

    # --- 3. Unpack and display SHAP IMPORTANCE ---
    # This assumes 'shap_importance' and 'feature_names' are in the 'run_config' JSON
    display(HTML("<h4>Feature Importance (SHAP)</h4>"))
    try:
        # The data is in the 'shap_importance' column
        shap_dict = json.loads(run_data['shap_importance'])
        
        if 'values' in shap_dict and 'features' in shap_dict:
            shap_series = pd.Series(
                shap_dict['values'], 
                index=shap_dict['features'],
                name="Mean SHAP Value"
            )
            # Sort for better readability
            display(shap_series.sort_values(ascending=False).to_frame())
        else:
            print("'values' or 'features' keys not found in 'shap_importance' JSON.")

    except (json.JSONDecodeError, TypeError):
        print("Could not parse the 'shap_importance' column. It might be malformed or empty.")
        print("Raw content:", run_data['shap_importance'])


# --- Usage in your Jupyter Notebook cell ---

# Assuming 'results_df' is the DataFrame from Part 1

# Inspect the details of the first run (index 0)
inspect_run_details(results_df, 0)

# Inspect the details of another run (e.g., index 5)
# inspect_run_details(results_df, 5)

--------------------------------------------------


Timestamp: 2025-06-11 03:31:29.367323
--------------------------------------------------


,Value
accuracy,0.488798
kelly_fraction,-0.00676
mc_trend_strength,None
mc_up_prob,None
mc_down_prob,None
mc_ci,"[None, None]"
mc_simulated_slopes,[]


,probability
0,0.589469
1,0.353351
2,0.267792
3,0.515819
4,0.307403
5,0.238311
6,0.248812
7,0.232842
8,0.311587
9,0.553192


,Mean SHAP Value
OBV,0.458183
RSI14,0.331338
%D,0.294365
MACD_hist,0.251742
Return1,0.241770
Volume,0.214259
GARCH_vol,0.207744
%K,0.199229
MFI14,0.193726
ROC10,0.182105


In [9]:
import os
from pathlib import Path
from typing import Union, List, Tuple

def convert_files_to_text(
    root_dir: str,
    output_dir: str,
    include_subdirs: Union[bool, List[str]] = True,
    exclude_dirs: List[str] = None,
    extensions: Tuple[str, ...] = ('.py', '.yml', '.toml'),
    exclude_files: List[str] = None
):
    """
    Convert files to text with flexible directory processing options.
    
    Args:
        root_dir: Directory containing source files
        output_dir: Where to save text versions
        include_subdirs: True=all, False=none, List=specific subdirs + root
        exclude_dirs: Directories to always exclude
        extensions: File extensions to process
        exclude_files: Specific filenames to exclude (including extensions)
    """
    # Create output directory if needed
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Default exclude directories and files
    if exclude_dirs is None:
        exclude_dirs = [".venv", ".vscode", ".git", "__pycache__"]
    if exclude_files is None:
        exclude_files = [".python-version", ".ipynb"]
    if extensions is None:
        extensions = '*'
    
    # Prepare files to process
    files_to_process = []
    
    # Always include root directory files
    files_to_process.extend(Path(root_dir).glob('*'))
    
    # Add subdirectories if specified
    if isinstance(include_subdirs, list):
        for subdir in include_subdirs:
            subdir_path = Path(root_dir) / subdir
            if subdir_path.exists():
                files_to_process.extend(subdir_path.rglob('*'))
    elif include_subdirs is True:
        # Include all subdirectories recursively
        files_to_process.extend(Path(root_dir).rglob('*'))
    
    # Track output filenames to prevent overwrites
    output_files = set()
    
    # NEW: Handle wildcard extension logic
    process_all_files = False
    if extensions == '*' or extensions == ('*',) or extensions is None:
        process_all_files = True
    
    for filepath in files_to_process:
        # Skip directories and excluded files
        if (not filepath.is_file() or 
            any(excluded in filepath.parts for excluded in exclude_dirs) or
            filepath.name in exclude_files):
            continue
            
        # MODIFIED: Extension check logic
        if process_all_files or filepath.suffix.lower() in extensions:
            # Get relative path to maintain structure
            rel_path = filepath.relative_to(root_dir)
            
            # Create unique output filename that preserves original extension
            output_filename = f"{rel_path.stem}_{rel_path.suffix[1:]}.txt"
            output_path = Path(output_dir) / rel_path.with_name(output_filename)
            
            # Ensure we don't overwrite files
            if output_path in output_files:
                print(f"Warning: Skipping potential overwrite of {output_path}")
                continue
                
            output_files.add(output_path)
            
            # Ensure parent directory exists
            output_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Read and write the file
            try:
                with open(filepath, 'r', encoding='utf-8') as infile, \
                     open(output_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(infile.read())
                
                print(f"Converted: {filepath} → {output_path}")
            except Exception as e:
                print(f"Error processing {filepath}: {str(e)}")


# Example usage
if __name__ == "__main__":
    convert_files_to_text(
        root_dir="/workspaces/smart_dev/.devcontainer",
        output_dir="/workspaces/smart_dev/projects/Notebooks/Stocks/text_output",
        include_subdirs=[],
        extensions=None,
        exclude_dirs=[".venv", ".vscode", ".git", "__pycache__", ".ruff_cache", "data", "results"],
        exclude_files=[".python-version", "local_settings.py", "*.ipynb", "ruff_check.log"]  # Additional excludes
    )

Converted: /workspaces/smart_dev/.devcontainer/devcontainer.json → /workspaces/smart_dev/projects/Notebooks/Stocks/text_output/devcontainer_json.txt
Converted: /workspaces/smart_dev/.devcontainer/docker-compose.yml → /workspaces/smart_dev/projects/Notebooks/Stocks/text_output/docker-compose_yml.txt
Converted: /workspaces/smart_dev/.devcontainer/Dockerfile → /workspaces/smart_dev/projects/Notebooks/Stocks/text_output/Dockerfile_.txt


In [ ]:
# fig = make_subplots(rows=1, cols=2)
# 7.1 Price chart with predicted buy/sell signals
fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'))
fig1.add_trace(go.Scatter(x=df.index, y=df['Close'], name="Signal", mode="lines"))
# Mark signals in the test period
test_dates = test_df.index  # dates corresponding to X_test
# Align Transformer's test predictions to actual dates (account for window offset)
signal_start_idx = train_df.index[-1]  # last train date
signal_dates = df.index[train_size:]   # dates from start of test
signals = pd.Series(test_preds, index=signal_dates[:len(test_preds)])
# Plot buy signals (predicted up) and sell signals (predicted down)
buy_signals = signals[signals == 1].index
sell_signals = signals[signals == 0].index
fig1.add_trace(go.Scatter(x=buy_signals, y=df.loc[buy_signals, 'Close'], 
                          mode='markers', marker_symbol='triangle-up', 
                          marker_color='green', marker_size=10, name='Predicted Buy'))
fig1.add_trace(go.Scatter(x=sell_signals, y=df.loc[sell_signals, 'Close'], 
                          mode='markers', marker_symbol='triangle-down', 
                          marker_color='red', marker_size=10, name='Predicted Sell'))
fig1.update_layout(title=f"{symbol} Price with Model Signals", 
                   yaxis_title="Price", xaxis_title="Date")

# 7.2 Feature importance bar chart (top 10 features from SHAP)
top_features = feature_importance.head(10)[::-1]  # reverse for plotting (smallest to largest)
fig2 = go.Figure(go.Bar(x=top_features.values, y=top_features.index, orientation='h', marker_color='blue'))
fig2.update_layout(title="Top 10 Feature Importances (SHAP)", xaxis_title="Mean |SHAP|", yaxis_title="Feature")

# 7.3 Display the interactive charts
fig1.show()
fig2.show()
